<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/comment_bertopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install -U sentence-transformers
!pip install bertopic==0.6
!pip install emoji
!pip install advertools
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=3d02dac54d5b9bc3bcd18603b6f8f791bc995b9d142381675ed2e0b30d36fc1e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel

import sklearn as sk
import os
import nltk
from nltk.data import find

import seaborn as sns
import matplotlib.pyplot as plt

import re
import string
import advertools as adv


In [4]:
df = pd.read_csv("comments.csv")
df

,Business Unit Name,EVP,Facility Type,District,Comment Language,Original Comment Text,English Comment,Compound Sentiment Score,Overall Sentiment,spl_char_chk,deEmojify_text,emoji_present_flag,all_punct_chk,english_comment_length
0,05R-Denver,"Griffith, Kelly",OF,60,en,I can step down and go back to being a pharmac...,I can step down and go back to being a pharmac...,-0.6908,Negative,False,I can step down and go back to being a pharmac...,False,False,277
1,05R-Denver,"Griffith, Kelly",OF,60,en,I have worked more hours than I would have lik...,I have worked more hours than I would have lik...,-0.4005,Negative,False,I have worked more hours than I would have lik...,False,False,278
2,05R-Denver,"Griffith, Kelly",OF,60,en,"Corporate SLT ""Thank You"" for implementing “Li...","Corporate SLT ""Thank You"" for implementing “Li...",-0.0258,Negative,False,"Corporate SLT ""Thank You"" for implementing “Li...",False,False,461
3,05R-Denver,"Griffith, Kelly",OF,60,en,We have made incredible strides in the Denver ...,We have made incredible strides in the Denver ...,0.0258,Positive,False,We have made incredible strides in the Denver ...,False,False,296
4,05R-Denver,"Griffith, Kelly",OF,60,en,"Work on benefits, seems poorer every year.","Work on benefits, seems poorer every year.",0.0258,Positive,False,"Work on benefits, seems poorer every year.",False,False,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58232,United,NaN,NaN,NaN,en,This is an amazing company to work with. In ou...,This is an amazing company to work with. In ou...,0.9947,Positive,False,This is an amazing company to work with. In ou...,False,False,1068
58233,United,NaN,NaN,NaN,en,"ever since I started working here, I have neve...","ever since I started working here, I have neve...",0.9949,Positive,False,"ever since I started working here, I have neve...",False,False,896
58234,United,NaN,NaN,NaN,en,Very good at working with request for days off...,Very good at working with request for days off...,0.9971,Positive,False,Very good at working with request for days off...,True,False,3213
58235,United,NaN,NaN,NaN,en,"I love working at my job it’s my happy place, ...","I love working at my job it’s my happy place, ...",0.9977,Positive,False,"I love working at my job it’s my happy place, ...",False,False,781


In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', 
                 min_topic_size= 15)
headline_topics, probs = topic_model.fit_transform(df['deEmojify_text'])

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-MiniLM-L3-v2


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/1820 [00:00<?, ?it/s]

2023-04-28 17:18:55,046 - BERTopic - Transformed documents to Embeddings


In [ ]:
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

In [ ]:
a_topic = freq.iloc[1]["Topic"] # Select the 1st topic
topic_model.get_topic(a_topic) # Show the words and their c-TF-IDF scores

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=10)

In [ ]:
similar_topics, similarity = topic_model.find_topics("pay", top_n = 5)
similar_topics 

In [ ]:
freq[freq["Topic"].isin(similar_topics)]

In [ ]:
# Get the clusterer model, the clusters' tree and the clusters (topics ids)
clusterer = topic_model.hdbscan_model
tree = clusterer.condensed_tree_
clusters = tree._select_clusters()

# Get the ids of the most relevant documents (exemplars) associated with the topic at index idx
c_exemplars = topic_model.get_most_relevant_documents(clusters[idx], tree)